# Importing required libraries and datasets

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
movies_meta = pd.read_csv('data/movies_metadata.csv',
                          usecols = ['id','imdb_id','title'])

movies_meta.head(2)

,id,imdb_id,title
0,862,tt0114709,Toy Story
1,8844,tt0113497,Jumanji


In [3]:
links = pd.read_csv('data/links.csv')

links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [4]:
ratings_small = pd.read_csv('data/ratings_small.csv')

ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
links  = links.merge(ratings_small, on = 'movieId')

links.head(2)

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7,3.0,851866703
1,1,114709,862.0,9,4.0,938629179


In [6]:
links.rename(columns = {'tmdbId':'id'}, inplace = True)

In [7]:
movies_meta = movies_meta.drop([29503,35587,19730])

In [8]:
movies_meta['id'] = movies_meta['id'].astype('int64')

In [9]:
movies_meta = movies_meta.merge(links,on='id')

In [10]:
df = movies_meta[['userId', 'movieId', 'rating', 'title']]

df.head(3)

,userId,movieId,rating,title
0,7,1,3.0,Toy Story
1,9,1,4.0,Toy Story
2,13,1,5.0,Toy Story


In [11]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (df.
                     groupby(by = ['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})
                     [['title', 'totalRatingCount']])

movie_ratingCount.head()

,title,totalRatingCount
0,$9.99,3
1,'Neath the Arizona Skies,1
2,"'night, Mother",3
3,(500) Days of Summer,45
4,...And God Created Woman,1


In [12]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.0,Toy Story,247
1,9,1,4.0,Toy Story,247
2,13,1,5.0,Toy Story,247
3,15,1,2.0,Toy Story,247
4,19,1,3.0,Toy Story,247


In [13]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   8754.000
mean      11.406
std       24.549
min        1.000
25%        1.000
50%        3.000
75%       10.000
max      341.000
Name: totalRatingCount, dtype: float64


In [14]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,7,1,3.000,Toy Story,247
1,9,1,4.000,Toy Story,247
2,13,1,5.000,Toy Story,247
3,15,1,2.000,Toy Story,247
4,19,1,3.000,Toy Story,247


In [15]:
movie_features_df = rating_popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
28 Days Later,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
300,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


# Model building

In [17]:
from scipy.sparse import csr_matrix

# converting the matrix into array matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)


from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

# Model Recommendation

In [161]:
def recommend_movie(movie_name):
    
    distances, indices = model_knn.kneighbors(movie_features_df.loc[movie_name,:].values.reshape(1, -1), n_neighbors = 6)
    
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

In [162]:
recommend_movie('Avatar')

Recommendations for Toy Story:

1: Inception, with distance of 0.3663126779224054:
2: Iron Man, with distance of 0.36696298155661766:
3: The Dark Knight, with distance of 0.4246926245558492:
4: District 9, with distance of 0.43666774462351055:
5: Star Trek, with distance of 0.4385474244949342:


# Model Serialization

In [64]:
from pickle import dump

import bz2

In [68]:
dump(model_knn, open('model/knn_model.pkl', 'wb'))

dump(movie_features_df.to_dict(),open('model/collaborative_cosine_similarity.pkl','wb'))

# Model Deserialization

In [69]:
import numpy as np
from pickle import load

In [71]:
knn_model = load(open('model/knn_model.pkl', 'rb'))

In [72]:
knn_model

NearestNeighbors(algorithm='brute', metric='cosine')

In [73]:
df = load(open('model/collaborative_cosine_similarity.pkl', 'rb'))

In [76]:
df = pd.DataFrame(df)

In [159]:
df.head(1)

,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
10 Things I Hate About You,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
